In [5]:
ls data_one/PetImages

Cat/ Dog/


In [5]:
#### random fill data to train and validation folder with 80% ratio ######## 



import os
import shutil
from random import sample



# Define paths
base_dir = './data_one/PetImages'  # Replace with the path to your PetImages folder
classes = ['Cat', 'Dog']
train_dir = os.path.join("model_one", 'train')
validation_dir = os.path.join("model_one", 'validation')

# Split ratio
train_ratio = 0.8

# Make new dirs
for cl in classes:
    os.makedirs(os.path.join(train_dir, cl), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, cl), exist_ok=True)

# Move files
for cl in classes:
    # Source dir
    src_dir = os.path.join(base_dir, cl)
    all_files = os.listdir(src_dir)
    
    # Shuffle and split files
    train_files = sample(all_files, int(len(all_files) * train_ratio))
    validation_files = list(set(all_files) - set(train_files))
    
    # Move to train_dir
    for file_name in train_files:
        shutil.move(os.path.join(src_dir, file_name), os.path.join(train_dir, cl))
    
    # Move to validation_dir
    for file_name in validation_files:
        shutil.move(os.path.join(src_dir, file_name), os.path.join(validation_dir, cl))


In [9]:
#### Train !! ###


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up image directories
train_dir = './model_one/train'
validation_dir = './model_one/validation'

"""

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # Sigmoid because we have two classes
])

"""

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model



# Image augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

# Load the base model, pre-trained on ImageNet
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional base
base_model.trainable = False

# Create a new model on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)




model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## Hyper parameter
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = validation_generator.samples // validation_generator.batch_size
print(f"BaschSize {train_generator.batch_size} and we get epoch {steps_per_epoch}, steps{validation_steps}")
"""


"""


# Train the model
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=15, validation_data=validation_generator, validation_steps=validation_steps)

# Save the model
model.save('cats_and_dogs_classifier.h5')





Found 19966 images belonging to 2 classes.
Found 4993 images belonging to 2 classes.
BaschSize 20 and we get epoch 998, steps249
Epoch 1/15
494/998 [=============>................] - ETA: 5:36 - loss: 0.7153 - accuracy: 0.5023

/Users/johnny_hung/Documents/ja_interview_entretien/ja_git/myenv/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


998/998 [==============================] - 820s 821ms/step - loss: 0.7045 - accuracy: 0.5008 - val_loss: 0.6931 - val_accuracy: 0.5014
Epoch 2/15
998/998 [==============================] - 776s 778ms/step - loss: 0.6934 - accuracy: 0.4976 - val_loss: 0.6931 - val_accuracy: 0.4996
Epoch 3/15
998/998 [==============================] - 776s 778ms/step - loss: 0.6935 - accuracy: 0.4959 - val_loss: 0.6932 - val_accuracy: 0.4990
Epoch 4/15
998/998 [==============================] - 776s 777ms/step - loss: 0.6933 - accuracy: 0.4966 - val_loss: 0.6932 - val_accuracy: 0.5004
Epoch 5/15
998/998 [==============================] - 772s 773ms/step - loss: 0.6933 - accuracy: 0.4964 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/15
998/998 [==============================] - 770s 771ms/step - loss: 0.6933 - accuracy: 0.5003 - val_loss: 0.6931 - val_accuracy: 0.5008
Epoch 7/15
998/998 [==============================] - 771s 773ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6932 - val_accura

In [1]:
## loaidng information
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('cats_and_dogs_classifier.h5')


# Print the model architecture
model.summary()

# Get input and output of the model
print("Input shape:", model.input_shape)
print("Output shape:", model.output_shape)


2024-04-22 13:18:27.959315: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 13:18:28.002224: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd33d6d3510 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-22 13:18:28.002253: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling_4 (Rescaling)         (None, 150, 150, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
normalization_4 (Normalization) (None, 150, 150, 3)  7           rescaling_4[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization_4[0][0]            
_______________________________________________________________________________________

In [11]:
## GET Class
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you used ImageDataGenerator for training
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    'model_one/train',  # path to the directory of training data
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

# Print class indices
print(train_generator.class_indices)




Found 19966 images belonging to 2 classes.
{'Cat': 0, 'Dog': 1}


In [14]:
## Loading test

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = load_model('cats_and_dogs_classifier.h5')

# Define the path to the new image
image_path = 'temp_load/test_chat.jpg'

# Load the image
img = image.load_img(image_path, target_size=(150, 150))

# Preprocess the image
img_tensor = image.img_to_array(img)  # Convert the image to a numpy array
img_tensor = np.expand_dims(img_tensor, axis=0)  # Add batch dimension
img_tensor /= 255.  # Model expects input in the range [0, 1]

# Predict the class of the image
prediction = model.predict(img_tensor)

# Interpret the output
# Assuming the output of the model is binary:
# '0' is for 'Cat' and '1' is for 'Dog'
print(f"prediciotn is {prediction}")
if prediction < 0.5:
    print(f"The image with path {image_path} is a Cat.")
else:
    print(f"The image with path {image_path} is a Dog.")


prediciotn is [[0.4987722]]
The image with path temp_load/test_chat.jpg is a Cat.


In [2]:
#import tensorflow as tf

## Breaking Down image loaing

In [31]:
# Step one: 150, 150, 3
# Define the path to the new image
image_path = 'temp_load/test_chat.jpg'

# Load the image
img = image.load_img(image_path, target_size=(150, 150))
img_tensor = image.img_to_array(img)
print(f"Shape >> {img_tensor.shape}")

img_tensor[149][149][2]


Shape >> (150, 150, 3)


189.0

In [32]:
# Step Two: Add Batch example
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor.shape

(1, 150, 150, 3)

In [13]:
## Step scal [0,1]
img_tensor /= 255. 
img_tensor

array([[[[0.00387543, 0.00392157, 0.00392157],
         [0.00392157, 0.00392157, 0.00389081],
         [0.00392157, 0.00392157, 0.00389081],
         ...,
         [0.00287582, 0.00236832, 0.00210688],
         [0.00312188, 0.00278354, 0.00258362],
         [0.00392157, 0.00387543, 0.00379854]],

        [[0.00387543, 0.00392157, 0.00392157],
         [0.00392157, 0.00392157, 0.00389081],
         [0.00392157, 0.00392157, 0.00389081],
         ...,
         [0.00298347, 0.00247597, 0.00221453],
         [0.00309112, 0.00272203, 0.00253749],
         [0.00392157, 0.0038293 , 0.00378316]],

        [[0.00387543, 0.00392157, 0.00392157],
         [0.00392157, 0.00392157, 0.00389081],
         [0.00392157, 0.00392157, 0.00389081],
         ...,
         [0.00298347, 0.00247597, 0.00218378],
         [0.00319877, 0.00282968, 0.00264514],
         [0.00392157, 0.00386005, 0.0038293 ]],

        ...,

        [[0.00316801, 0.00301423, 0.00282968],
         [0.00295271, 0.00275279, 0.00250673]

## About history

In [1]:
import json
# Write
"""
# Convert the history.history dict to a JSON file
with open('model_history.json', 'w') as f:
    json.dump(history.history, f)
"""

## Load
# Load history from JSON
with open('model_history.json', 'r') as f:
    loaded_history = json.load(f)

print(loaded_history)



{'loss': [0.7193088531494141, 0.6946874856948853, 0.6871427893638611, 0.6872152090072632, 0.6900128722190857, 0.6850082874298096, 0.6849346160888672, 0.6863163113594055, 0.6850281357765198, 0.677954912185669, 0.6882190704345703, 0.6764642000198364, 0.674538254737854, 0.6645799279212952, 0.6728416085243225], 'accuracy': [0.5270000100135803, 0.49549999833106995, 0.5605000257492065, 0.559499979019165, 0.546500027179718, 0.5720040202140808, 0.5513595342636108, 0.5599194169044495, 0.5684999823570251, 0.5755000114440918, 0.5629405975341797, 0.5964999794960022, 0.597000002861023, 0.6110000014305115, 0.5954999923706055], 'val_loss': [0.7069425582885742, 0.686040997505188, 0.6783363819122314, 0.6833188533782959, 0.6895650029182434, 0.6856113076210022, 0.6807612776756287, 0.6834836602210999, 0.6620160341262817, 0.6566139459609985, 0.6546926498413086, 0.6858153939247131, 0.6606965065002441, 0.6524175405502319, 0.6486132144927979], 'val_accuracy': [0.5009999871253967, 0.5709999799728394, 0.5970000

In [41]:
##
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


ImportError: Matplotlib requires numpy>=1.20; you have 1.18.5